<a href="https://colab.research.google.com/github/dldbwls00/NLP-sentiment-analysis-project/blob/main/kobert_sentiment_lyrics_0123ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

kobert finetuning

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# package install
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install tokenizers
!pip install torch
!pip install pandas
!pip install transformers==4.30.0
!pip install gluonnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 27.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661769 sha256=0e086bded8527453608a41ba233694bcac88e599a753631d5d72154fb84c148c
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uni

In [3]:
# kobert install
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-wki26xmq/kobert-tokenizer_dc2a5cd0cf0b4b3bbe4ebd64c638f4ec
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-wki26xmq/kobert-tokenizer_dc2a5cd0cf0b4b3bbe4ebd64c638f4ec
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=22293585fe74fe4b607685224966caaebf46eeed47eda773488d199ebf26ddf4
  Stored in directory: /tmp/pip-ephem-wheel-cache-s0lmkznn/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [5]:
# importing kobert using hugging face
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
# gpu setting
device = torch.device("cuda:0")
!nvidia-smi
# device = torch.device("cpu")

Tue Jan 23 08:42:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
# tokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [8]:
# connecting google drive - colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# loading dataset
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/dataset/lyrics.xlsx')

In [10]:
print(len(df))
df.sample(n=5)

5470


,Unnamed: 0,Song_ID,Song_Name,Artist,Emotion,Lyrics
5256,5256,31834636,물바다,정재일,fear,가사를 찾을 수 없습니다.
900,900,36456696,한사람만 (2023) (여름날 우리 X 이홍기 (FT아일랜드)),이홍기 (FT아일랜드),love,한 여자밖에 모르고 한 여자만을 사랑합니다 그게 부담될 줄 모르고 떠날 이유를 준 ...
465,465,34747275,MY GIRL (for FAN),TAN(티에이엔),love,다 알아 버렸어 알기 쉬운 너 미칠 것 같아 I’m so crazy 겉은 멀쩡한데 ...
3473,3473,31605780,상수역,검정치마,sad,누군가 궁금한 적 있다면 난 늦은 밤 상수역만 맴돌았죠 왜냐고는 내게 묻지 말아...
1456,1456,35380366,_WORLD,세븐틴 (SEVENTEEN),joy,Hey 아까부터 널 봤어 우린 처음이지만 모든 재미를 느낄 수 있어 더 알고 싶어 ...


In [11]:
# 'Song_ID' 컬럼을 기준으로 중복된 행 제거
df = df.drop_duplicates(subset='Song_ID', keep='first')

In [12]:
# 'Lyrics' 열이 '가사를 찾을 수 없습니다.'인 행 삭제
df = df[df['Lyrics'] != '가사를 찾을 수 없습니다.']

In [13]:
# 'Unnamed: 0' 열이 숫자인 행만 선택
df = df[pd.to_numeric(df['Unnamed: 0'], errors='coerce').notna()]

In [14]:
# 제거할 Song_ID 리스트
songs_to_remove = [33635302, 32816089, 36593603, 35498322, 34657617, 30295465, 36406020]

# Song_ID가 제거할 대상이 아닌 행만 선택
df = df[~df['Song_ID'].isin(songs_to_remove)]

In [15]:
# 제거할 Song_ID 리스트
songs_to_remove = [7940250]

# Song_ID가 제거할 대상이 아닌 행만 선택
df = df[~df['Song_ID'].isin(songs_to_remove)]

In [16]:
# 'Unnamed: 0' 열 삭제
song = df.drop(['Unnamed: 0', 'Song_ID', 'Song_Name', 'Artist' ], axis=1)

In [17]:
song.loc[(song['Emotion']=='joy'), 'Emotion'] = 0  #기쁨
song.loc[(song['Emotion']=='love'), 'Emotion'] = 1  #사랑
song.loc[(song['Emotion']=='happy'), 'Emotion'] = 0 #기쁨
song.loc[(song['Emotion']=='angry'), 'Emotion'] = 2 #분노
song.loc[(song['Emotion']=='fear'), 'Emotion'] = 3 #불안
song.loc[(song['Emotion']=='wound'), 'Emotion'] = 4 #상처
song.loc[(song['Emotion']=='sad'), 'Emotion'] = 5 #슬픔
song.loc[(song['Emotion']=='lonely'), 'Emotion'] = 6 #외로움

In [18]:
# Lyrics 열의 내용을 띄어쓰기 단위로 나누어 새로운 행 생성
max_length = 40  # 띄어쓰기 단위로 나눌 최대 길이

# 새로운 데이터를 저장할 딕셔너리
new_data = {
    'Emotion': [],       # 감정 정보를 저장할 리스트
    'Lyrics_chunk': []   # 띄어쓰기 단위로 나뉜 부분을 저장할 리스트
}

# 주어진 데이터프레임(song)의 각 행에 대해 반복
for _, row in song.iterrows():  # DataFrame의 각 행의 정보를 담은 객체
    emotion = row['Emotion']    # 현재 행의 감정 정보
    lyrics = row['Lyrics']      # 현재 행의 가사 정보
    # 띄어쓰기 단위로 나눈 부분을 lyrics_chunks에 저장
    lyrics_chunks = [lyrics[i:i+max_length] for i in range(0, len(lyrics), max_length) if ' ' in lyrics[i:i+max_length]]

    # 각 띄어쓰기 단위로 나뉜 부분을 새로운 데이터에 추가
    for chunk in lyrics_chunks:
        new_data['Emotion'].append(emotion)
        new_data['Lyrics_chunk'].append(chunk)

# 새로운 데이터프레임 생성
song = pd.DataFrame(new_data)

In [19]:
song

,Emotion,Lyrics_chunk
0,1,꿈인 것 같아 너를 만난 건 내 마음속에 빛으로 내려와 나의 두 눈에 너
1,1,를 담을 거야 영원히 널 내 곁에 지켜갈게 너를 위한 시간을 내고 이야길
2,1,듣고 너의 입술에 미소를 줄게 세상은 다 너로 가득해 너를 닮아가 어딜
3,1,봐도 다 너야 It's you yeah 하루도 빠짐없이 보는데도 보고
4,1,싶어 널 옆에 두고 물어봐 지금 어디 있어 어제 오늘 내일 봄 여름 가을
...,...,...
43722,3,제보다 커진 내 방안에 하얗게 밝아온 유리창엔 썼다 지운다 널 사랑해
43723,3,밤 하늘에 빛나는 수많은 별들 저마다 아름답지만 내 맘속에 빛나는
43724,3,별 하나 오직 너만 있을 뿐이야 창틈에 기다리던 새벽이 오면 어제
43725,3,보다 커진 내 방안에 하얗게 밝아온 유리창에 썼다 지운다 널 사랑해


In [22]:
# extract column -> to array
data_list = []
for q, label in zip(song['Lyrics_chunk'], song['Emotion']) :
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [23]:
# array check
print(len(data_list))
print(data_list[0])
print(data_list[100])
print(data_list[200])
print(data_list[500])
print(data_list[-1])

43727
['꿈인 것 같아 너를 만난 건 내 마음속에 빛으로 내려와 나의 두 눈에 너', '1']
[' 알 것 같아 궁금해 넌 어때 어서 대답해봐 어떻게 생각해 지금 고백하는', '1']
['지 사랑시 고백구 행복동  그대를 만나고 내가 사는 이곳 별빛보다 더 아', '1']
['했던 내 일상이 편안해져만 가 내 맘은 보랏빛 물결 눈만 떠도 웃음이나 ', '1']
[' 하얗게 밝아온 유리창에  썼다 지운다 널 사랑해 -  ', '3']


In [24]:
# splitting data
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(len(dataset_train))
print(len(dataset_test))

32795
10932


In [25]:
"""class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
"""


class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [26]:
# setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [27]:
# preprocessing
tok = tokenizer.tokenize
#tokenizer = get_tokenizer()
#tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [28]:
# kobert training model
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [29]:
# load model
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [30]:
# prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [31]:
# model training
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-31-4b508a570d4d>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/513 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9504530429840088 train acc 0.234375
epoch 1 batch id 201 loss 1.700021505355835 train acc 0.25590796019900497
epoch 1 batch id 401 loss 1.5871332883834839 train acc 0.29636845386533667
epoch 1 train acc 0.3144820319832503


<ipython-input-31-4b508a570d4d>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/171 [00:00<?, ?it/s]

...

epoch 4 test acc 0.5453216374269005


  0%|          | 0/513 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.6833009719848633 train acc 0.734375
epoch 5 batch id 201 loss 0.7872524261474609 train acc 0.7351523631840796
epoch 5 batch id 401 loss 0.7137659192085266 train acc 0.7419342269326683
epoch 5 train acc 0.7440121651866292


  0%|          | 0/171 [00:00<?, ?it/s]

epoch 5 test acc 0.5491593567251462


In [32]:
# test
#tokenizer = get_tokenizer()
#tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = tokenizer.tokenize

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨")
            elif np.argmax(logits) == 1:
                test_eval.append("사랑")
            elif np.argmax(logits) == 2:
                test_eval.append("분노")
            elif np.argmax(logits) == 3:
                test_eval.append("불안")
            elif np.argmax(logits) == 4:
                test_eval.append("상처")
            elif np.argmax(logits) == 5:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 5:
                test_eval.append("외로움")

        print(test_eval[0])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [33]:
# test loop: 0 to quit
"""end = 1
while end == 1 :
    sentence = input("write sentences: ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")"""

'end = 1\nwhile end == 1 :\n    sentence = input("write sentences: ")\n    if sentence == 0 :\n        break\n    predict(sentence)\n    print("\n")'